<a href="https://colab.research.google.com/github/GireeshRavula/Coding-Raja-Technologies-Internship/blob/main/movie_recommendations_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
movies = pd.read_csv("/content/movies.csv")

In [2]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
import re

def clean_title(title):
    title = re.sub("[^a-zA-Z0-9 ]", "", title)
    return title

In [4]:
movies["clean_title"] = movies["title"].apply(clean_title)

In [5]:
movies

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995
...,...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy,Kein Bund frs Leben 2007
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,Feuer Eis Dosenbier 2002
27275,131258,The Pirates (2014),Adventure,The Pirates 2014
27276,131260,Rentun Ruusu (2001),(no genres listed),Rentun Ruusu 2001


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(movies["clean_title"])

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = movies.iloc[indices].iloc[::-1]

    return results

In [8]:
import ipywidgets as widgets
from IPython.display import display

movie_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
movie_list = widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            display(search(title))

movie_input.observe(on_type, names='value')


display(movie_input, movie_list)

Text(value='Toy Story', description='Movie Title:')

Output()

In [9]:
movie_id = 89745
movie = movies[movies["movieId"] == str(movie_id)]

In [10]:
ratings = pd.read_csv("/content/ratings.csv")

In [11]:
ratings.dtypes

,0
userId,int64
movieId,float64
rating,float64
timestamp,float64


In [12]:
similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()

In [13]:
similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]

In [14]:
similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

similar_user_recs = similar_user_recs[similar_user_recs > .10]

In [15]:
all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]


In [16]:
all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

In [17]:
rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
rec_percentages.columns = ["similar", "all"]

In [18]:
rec_percentages

,similar,all
movieId,,
89745.0,1.000000,0.015314
58559.0,0.607261,0.090620
79132.0,0.570957,0.055949
7153.0,0.485149,0.131608
2571.0,0.468647,0.213939
...,...,...
26614.0,0.102310,0.011068
59784.0,0.102310,0.009855
30707.0,0.102310,0.024057


In [19]:
rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]

In [20]:
rec_percentages = rec_percentages.sort_values("score", ascending=False)

In [21]:
rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")

,similar,all,score,movieId,title,genres,clean_title
17874,1.000000,0.015314,65.300330,89745,"Avengers, The (2012)",Action|Adventure|Sci-Fi|IMAX,Avengers The 2012
23085,0.148515,0.003184,46.643093,110102,Captain America: The Winter Soldier (2014),Action|Adventure|Sci-Fi|IMAX,Captain America The Winter Soldier 2014
20906,0.188119,0.004043,46.526485,102125,Iron Man 3 (2013),Action|Sci-Fi|Thriller|IMAX,Iron Man 3 2013
17040,0.145215,0.003437,42.253155,86332,Thor (2011),Action|Adventure|Drama|Fantasy|IMAX,Thor 2011
17506,0.145215,0.003487,41.640790,88140,Captain America: The First Avenger (2011),Action|Adventure|Sci-Fi|Thriller|War,Captain America The First Avenger 2011
19204,0.118812,0.002982,39.844269,95510,"Amazing Spider-Man, The (2012)",Action|Adventure|Sci-Fi|IMAX,Amazing SpiderMan The 2012
18349,0.115512,0.002982,38.737484,91630,Mission: Impossible - Ghost Protocol (2011),Action|Adventure|Thriller|IMAX,Mission Impossible Ghost Protocol 2011
23381,0.132013,0.003639,36.277961,111362,X-Men: Days of Future Past (2014),Action|Adventure|Sci-Fi,XMen Days of Future Past 2014
18318,0.188119,0.005206,36.137076,91542,Sherlock Holmes: A Game of Shadows (2011),Action|Adventure|Comedy|Crime|Mystery|Thriller,Sherlock Holmes A Game of Shadows 2011
20994,0.171617,0.004801,35.743339,102445,Star Trek Into Darkness (2013),Action|Adventure|Sci-Fi|IMAX,Star Trek Into Darkness 2013


In [22]:
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]

    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]

In [23]:
import ipywidgets as widgets
from IPython.display import display

movie_name_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title:')

Output()